# Stream Ensemble Classification
---

## `NEWeather` dataset

**Description:** The National Oceanic and Atmospheric Administration (NOAA),
has compiled a database of weather measurements from over 7,000 weather 
stations worldwide. Records date back to the mid-1900s. Daily measurements
include a variety of features (temperature, pressure, wind speed, etc.) as
well as a series of indicators for precipitation and other weather-related
events. The `NEweather` dataset contains data from this database, specifically
from the Offutt Air Force Base in Bellevue, Nebraska ranging for over 50 years
(1949-1999).

**Features:** 8 Daily weather measurements
 
|       Attribute      | Description |
|:--------------------:|:-----------------------------|
| `temp`                   | Temperature
| `dew_pnt`                | Dew Point
| `sea_lvl_press`          | Sea Level Pressure
| `visibility`             | Visibility
| `avg_wind_spd`           | Average Wind Speed
| `max_sustained_wind_spd` | Maximum Sustained Wind Speed
| `max_temp`               | Maximum Temperature
| `min_temp`               | Minimum Temperature


**Class:** `rain` | 0: no rain, 1: rain
 
**Samples:** 18,159


In [1]:
import pandas as pd
from river.stream import iter_pandas
from river.metrics import Metrics,Accuracy,BalancedAccuracy,CohenKappa,GeometricMean,Rolling
from river.evaluate import progressive_val_score

In [2]:
data = pd.read_csv("../datasets/NEweather.csv")
features = data.columns[:-1]

## Online Bagging

---
[Online Bagging](https://riverml.xyz/0.10.1/api/ensemble/BaggingClassifier/) is an Online bootstrap aggregation for classification.

In [3]:
from river.ensemble import BaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = BaggingClassifier(model=HoeffdingTreeClassifier(),
                          n_models=10,
                          seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 70.77%, BalancedAccuracy: 67.47%, GeometricMean: 66.80%, CohenKappa: 34.45%
[2,000] Accuracy: 70.29%, BalancedAccuracy: 66.94%, GeometricMean: 66.42%, CohenKappa: 32.37%
[3,000] Accuracy: 71.29%, BalancedAccuracy: 62.56%, GeometricMean: 59.04%, CohenKappa: 26.45%
[4,000] Accuracy: 71.82%, BalancedAccuracy: 61.73%, GeometricMean: 56.61%, CohenKappa: 25.72%
[5,000] Accuracy: 72.67%, BalancedAccuracy: 62.70%, GeometricMean: 57.67%, CohenKappa: 27.97%
[6,000] Accuracy: 73.18%, BalancedAccuracy: 63.95%, GeometricMean: 59.71%, CohenKappa: 30.34%
[7,000] Accuracy: 74.10%, BalancedAccuracy: 65.01%, GeometricMean: 61.06%, CohenKappa: 32.60%
[8,000] Accuracy: 73.83%, BalancedAccuracy: 64.93%, GeometricMean: 61.02%, CohenKappa: 32.39%
[9,000] Accuracy: 73.95%, BalancedAccuracy: 64.68%, GeometricMean: 60.30%, CohenKappa: 32.20%
[10,000] Accuracy: 74.11%, BalancedAccuracy: 64.59%, GeometricMean: 60.03%, CohenKappa: 32.14%
[11,000] Accuracy: 74.56%, BalancedAccuracy: 64.71%, Geomet

Accuracy: 74.68%, BalancedAccuracy: 66.38%, GeometricMean: 62.53%, CohenKappa: 35.78%

## Leveraging Bagging
---

[Leveraging Bagging](https://riverml.xyz/0.10.1/api/ensemble/LeveragingBaggingClassifier/) is an improvement over the Oza Bagging algorithm. The bagging performance is leveraged by increasing the re-sampling. It uses a poisson distribution to simulate the re-sampling process. To increase re-sampling it uses a higher w value of the Poisson distribution (agerage number of events), 6 by default, increasing the input space diversity, by attributing a different range of weights to the data samples.

To deal with concept drift, Leveraging Bagging uses the ADWIN algorithm to monitor the performance of each member of the enemble If concept drift is detected, the worst member of the ensemble (based on the error estimation by ADWIN) is replaced by a new (empty) classifier.

In [4]:
from river.ensemble import LeveragingBaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = LeveragingBaggingClassifier(model=HoeffdingTreeClassifier(),
                          n_models=10,
                          seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 76.48%, BalancedAccuracy: 73.05%, GeometricMean: 72.38%, CohenKappa: 46.29%
[2,000] Accuracy: 78.04%, BalancedAccuracy: 71.86%, GeometricMean: 70.18%, CohenKappa: 45.52%
[3,000] Accuracy: 78.53%, BalancedAccuracy: 70.76%, GeometricMean: 68.33%, CohenKappa: 44.29%
[4,000] Accuracy: 78.02%, BalancedAccuracy: 69.92%, GeometricMean: 67.08%, CohenKappa: 42.99%
[5,000] Accuracy: 77.88%, BalancedAccuracy: 69.91%, GeometricMean: 67.10%, CohenKappa: 42.92%
[6,000] Accuracy: 77.81%, BalancedAccuracy: 70.48%, GeometricMean: 68.09%, CohenKappa: 43.60%
[7,000] Accuracy: 78.25%, BalancedAccuracy: 71.20%, GeometricMean: 69.05%, CohenKappa: 44.86%
[8,000] Accuracy: 77.85%, BalancedAccuracy: 70.79%, GeometricMean: 68.56%, CohenKappa: 44.05%
[9,000] Accuracy: 77.75%, BalancedAccuracy: 70.38%, GeometricMean: 67.88%, CohenKappa: 43.57%
[10,000] Accuracy: 77.98%, BalancedAccuracy: 70.44%, GeometricMean: 67.87%, CohenKappa: 43.83%
[11,000] Accuracy: 78.14%, BalancedAccuracy: 70.44%, Geomet

Accuracy: 78.24%, BalancedAccuracy: 72.22%, GeometricMean: 70.38%, CohenKappa: 46.79%

## AdaptiveRandomForest
---



The 3 most important aspects of [ARF](https://riverml.xyz/0.10.1/api/ensemble/AdaptiveRandomForestClassifier/) are:
- inducing diversity through re-sampling
- inducing diversity through randomly selecting subsets of features for node splits
- drift detectors per base tree, which cause selective resets in response to drifts

It also allows training background trees, which start training if a warning is detected and replace the active tree if the warning escalates to a drift.

In [5]:
from river.ensemble import AdaptiveRandomForestClassifier

model = AdaptiveRandomForestClassifier(n_models=10)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 74.37%, BalancedAccuracy: 66.35%, GeometricMean: 62.20%, CohenKappa: 35.94%
[2,000] Accuracy: 76.84%, BalancedAccuracy: 67.23%, GeometricMean: 62.80%, CohenKappa: 38.44%
[3,000] Accuracy: 77.33%, BalancedAccuracy: 66.70%, GeometricMean: 61.77%, CohenKappa: 37.72%
[4,000] Accuracy: 77.37%, BalancedAccuracy: 67.05%, GeometricMean: 62.14%, CohenKappa: 38.53%
[5,000] Accuracy: 77.46%, BalancedAccuracy: 67.50%, GeometricMean: 62.87%, CohenKappa: 39.34%
[6,000] Accuracy: 77.45%, BalancedAccuracy: 68.11%, GeometricMean: 64.06%, CohenKappa: 40.21%
[7,000] Accuracy: 78.08%, BalancedAccuracy: 69.10%, GeometricMean: 65.47%, CohenKappa: 42.10%
[8,000] Accuracy: 77.68%, BalancedAccuracy: 69.01%, GeometricMean: 65.53%, CohenKappa: 41.69%
[9,000] Accuracy: 77.62%, BalancedAccuracy: 68.71%, GeometricMean: 64.94%, CohenKappa: 41.34%
[10,000] Accuracy: 77.74%, BalancedAccuracy: 68.66%, GeometricMean: 64.78%, CohenKappa: 41.33%
[11,000] Accuracy: 78.00%, BalancedAccuracy: 68.80%, Geomet

Accuracy: 78.04%, BalancedAccuracy: 70.70%, GeometricMean: 67.89%, CohenKappa: 44.80%

## StreamingRandomPatches
---
[SRP](https://riverml.xyz/0.10.1/api/ensemble/SRPClassifier/) is an ensemble method that simulates bagging or random subspaces. The default algorithm uses both bagging and random subspaces, namely Random Patches. The default base estimator is a Hoeffding Tree, but other base estimators can be used (differently from random forest variations).

In [6]:
from river.ensemble import SRPClassifier
from river.tree import HoeffdingTreeClassifier

model = SRPClassifier(model=HoeffdingTreeClassifier(),
                      n_models=10,
                      seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metrics, 
                      print_every=1000)

[1,000] Accuracy: 74.77%, BalancedAccuracy: 67.28%, GeometricMean: 63.73%, CohenKappa: 37.59%
[2,000] Accuracy: 76.49%, BalancedAccuracy: 66.64%, GeometricMean: 61.95%, CohenKappa: 37.27%
[3,000] Accuracy: 77.89%, BalancedAccuracy: 67.07%, GeometricMean: 61.97%, CohenKappa: 38.83%
[4,000] Accuracy: 77.99%, BalancedAccuracy: 67.64%, GeometricMean: 62.74%, CohenKappa: 40.01%
[5,000] Accuracy: 78.06%, BalancedAccuracy: 67.94%, GeometricMean: 63.20%, CohenKappa: 40.59%
[6,000] Accuracy: 77.93%, BalancedAccuracy: 68.11%, GeometricMean: 63.60%, CohenKappa: 40.72%
[7,000] Accuracy: 78.37%, BalancedAccuracy: 68.79%, GeometricMean: 64.63%, CohenKappa: 42.02%
[8,000] Accuracy: 77.95%, BalancedAccuracy: 68.55%, GeometricMean: 64.42%, CohenKappa: 41.38%
[9,000] Accuracy: 77.90%, BalancedAccuracy: 68.42%, GeometricMean: 64.12%, CohenKappa: 41.26%
[10,000] Accuracy: 78.09%, BalancedAccuracy: 68.56%, GeometricMean: 64.27%, CohenKappa: 41.58%
[11,000] Accuracy: 78.34%, BalancedAccuracy: 68.67%, Geomet

Accuracy: 78.37%, BalancedAccuracy: 70.53%, GeometricMean: 67.31%, CohenKappa: 44.97%

## Concept Drift Impact

Concept drift can negatively impact learning methods if not properly handled. Multiple real-world applications suffer **model degradation** as the models can not adapt to changes in the data.

---
## `AGRAWAL` dataset

We will load the data from a csv file. The data was generated using the `AGRAWAL` data generator with 3 **gradual drifts** at the 5k, 10k, and 15k marks. It contains 9 features, 6 numeric and 3 categorical.

There are 10 functions for generating binary class labels from the features. These functions determine whether a **loan** should be approved.

| Feature    | Description            | Values                                                                |
|------------|------------------------|-----------------------------------------------------------------------|
| `salary`     | salary                 | uniformly distributed from 20k to 150k                                |
| `commission` | commission             | if (salary <   75k) then 0 else uniformly distributed from 10k to 75k |
| `age`        | age                    | uniformly distributed from 20 to 80                                   |
| `elevel`     | education level        | uniformly chosen from 0 to 4                                          |
| `car`        | car maker              | uniformly chosen from 1 to 20                                         |
| `zipcode`    | zip code of the town   | uniformly chosen from 0 to 8                                          |
| `hvalue`     | value of the house     | uniformly distributed from 50k x zipcode to 100k x zipcode            |
| `hyears`     | years house owned      | uniformly distributed from 1 to 30                                    |
| `loan`       | total loan amount      | uniformly distributed from 0 to 500k                                  |

**Class:** `y` | 0: no loan, 1: loan
 
**Samples:** 20,000

`elevel`, `car`, and `zipcode` are categorical features.

In [7]:
data = pd.read_csv("../datasets/agr_a_20k.csv")
features = data.columns[:-1]

Since there are several drifts, to better track the model behaviour, let's use a [Rolling Window](https://riverml.xyz/0.10.1/api/metrics/Rolling/) to compute the performance. It is a fixed size wrapper for computing metrics ovear a window. When the window is full, the oldest element is removed. In this way, the performance refers to the most recent data, possibly affected by a concept drift

## ADWIN Online Bagging

---
[ADWIN Online Bagging](https://riverml.xyz/0.10.1/api/ensemble/ADWINBaggingClassifier/) is the online bagging method with the addition of the ADWIN algorithm as a change detector. If concept drift is detected, the worst member of the ensemble (based on the error estimation by ADWIN) is replaced by a new (empty) classifier.

In [8]:
from river.ensemble import ADWINBaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = ADWINBaggingClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                          n_models=10,
                          seed=42)
#metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
metrics = Rolling(metric=Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()]),window_size=500)
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=500)

[500] Accuracy: 79.16%, BalancedAccuracy: 71.24%, GeometricMean: 67.10%, CohenKappa: 47.58%	(rolling 500)
[1,000] Accuracy: 84.00%, BalancedAccuracy: 76.81%, GeometricMean: 73.77%, CohenKappa: 59.78%	(rolling 500)
[1,500] Accuracy: 88.40%, BalancedAccuracy: 86.09%, GeometricMean: 85.72%, CohenKappa: 74.05%	(rolling 500)
[2,000] Accuracy: 93.00%, BalancedAccuracy: 93.27%, GeometricMean: 93.27%, CohenKappa: 84.73%	(rolling 500)
[2,500] Accuracy: 89.20%, BalancedAccuracy: 87.31%, GeometricMean: 87.14%, CohenKappa: 75.10%	(rolling 500)
[3,000] Accuracy: 89.60%, BalancedAccuracy: 88.16%, GeometricMean: 88.06%, CohenKappa: 76.55%	(rolling 500)
[3,500] Accuracy: 91.00%, BalancedAccuracy: 90.61%, GeometricMean: 90.61%, CohenKappa: 79.37%	(rolling 500)
[4,000] Accuracy: 93.20%, BalancedAccuracy: 91.28%, GeometricMean: 91.13%, CohenKappa: 84.00%	(rolling 500)
[4,500] Accuracy: 87.00%, BalancedAccuracy: 84.39%, GeometricMean: 84.00%, CohenKappa: 70.31%	(rolling 500)
[5,000] Accuracy: 90.60%, Bala

Accuracy: 68.40%, BalancedAccuracy: 67.54%, GeometricMean: 67.44%, CohenKappa: 34.38%	(rolling 500)

## Leveraging Bagging

In [9]:
from river.ensemble import LeveragingBaggingClassifier
from river.tree import HoeffdingTreeClassifier

model = LeveragingBaggingClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                          n_models=10,
                          seed=42)
#metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
metrics = Rolling(metric=Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()]),window_size=500)
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=500)

[500] Accuracy: 70.14%, BalancedAccuracy: 62.38%, GeometricMean: 57.80%, CohenKappa: 26.97%	(rolling 500)
[1,000] Accuracy: 76.20%, BalancedAccuracy: 68.70%, GeometricMean: 64.98%, CohenKappa: 41.08%	(rolling 500)
[1,500] Accuracy: 81.60%, BalancedAccuracy: 76.42%, GeometricMean: 74.27%, CohenKappa: 56.84%	(rolling 500)
[2,000] Accuracy: 83.60%, BalancedAccuracy: 77.74%, GeometricMean: 75.55%, CohenKappa: 60.29%	(rolling 500)
[2,500] Accuracy: 83.60%, BalancedAccuracy: 78.28%, GeometricMean: 76.85%, CohenKappa: 60.10%	(rolling 500)
[3,000] Accuracy: 84.60%, BalancedAccuracy: 79.63%, GeometricMean: 78.21%, CohenKappa: 63.13%	(rolling 500)
[3,500] Accuracy: 86.80%, BalancedAccuracy: 81.81%, GeometricMean: 80.78%, CohenKappa: 67.26%	(rolling 500)
[4,000] Accuracy: 89.00%, BalancedAccuracy: 84.98%, GeometricMean: 84.27%, CohenKappa: 73.33%	(rolling 500)
[4,500] Accuracy: 86.80%, BalancedAccuracy: 83.37%, GeometricMean: 82.69%, CohenKappa: 69.35%	(rolling 500)
[5,000] Accuracy: 89.80%, Bala

Accuracy: 67.80%, BalancedAccuracy: 64.25%, GeometricMean: 62.47%, CohenKappa: 29.48%	(rolling 500)

## AdaptiveRandomForest

In [10]:
from river.ensemble import AdaptiveRandomForestClassifier

model = AdaptiveRandomForestClassifier(n_models=10,nominal_attributes=['elevel', 'car', 'zipcode'])
#metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
metrics = Rolling(metric=Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()]),window_size=500)
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=500)

[500] Accuracy: 65.53%, BalancedAccuracy: 56.98%, GeometricMean: 50.79%, CohenKappa: 15.29%	(rolling 500)
[1,000] Accuracy: 68.80%, BalancedAccuracy: 59.07%, GeometricMean: 51.49%, CohenKappa: 20.51%	(rolling 500)
[1,500] Accuracy: 73.40%, BalancedAccuracy: 66.16%, GeometricMean: 61.19%, CohenKappa: 35.78%	(rolling 500)
[2,000] Accuracy: 75.00%, BalancedAccuracy: 67.66%, GeometricMean: 63.64%, CohenKappa: 38.80%	(rolling 500)
[2,500] Accuracy: 73.00%, BalancedAccuracy: 62.80%, GeometricMean: 55.89%, CohenKappa: 29.28%	(rolling 500)
[3,000] Accuracy: 73.80%, BalancedAccuracy: 64.36%, GeometricMean: 57.74%, CohenKappa: 32.78%	(rolling 500)
[3,500] Accuracy: 78.60%, BalancedAccuracy: 69.58%, GeometricMean: 65.50%, CohenKappa: 43.83%	(rolling 500)
[4,000] Accuracy: 78.40%, BalancedAccuracy: 68.55%, GeometricMean: 63.10%, CohenKappa: 42.61%	(rolling 500)
[4,500] Accuracy: 75.20%, BalancedAccuracy: 66.50%, GeometricMean: 60.84%, CohenKappa: 37.32%	(rolling 500)
[5,000] Accuracy: 72.60%, Bala

Accuracy: 68.20%, BalancedAccuracy: 64.38%, GeometricMean: 62.30%, CohenKappa: 29.92%	(rolling 500)

## StreamingRandomPatches
---
We set the drift and warning detection options

In [11]:
from river.ensemble import SRPClassifier
from river.tree import HoeffdingTreeClassifier
from river.drift import ADWIN

model = SRPClassifier(model=HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode']),
                      n_models=10,
                      drift_detector=ADWIN(delta=0.001),
                      warning_detector=ADWIN(delta=0.01),
                      seed=42)
#metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
metrics = Rolling(metric=Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()]),window_size=500)
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metrics, 
                      print_every=500)

[500] Accuracy: 85.17%, BalancedAccuracy: 79.78%, GeometricMean: 78.09%, CohenKappa: 64.14%	(rolling 500)
[1,000] Accuracy: 93.40%, BalancedAccuracy: 91.88%, GeometricMean: 91.77%, CohenKappa: 84.91%	(rolling 500)
[1,500] Accuracy: 94.00%, BalancedAccuracy: 92.58%, GeometricMean: 92.45%, CohenKappa: 86.68%	(rolling 500)
[2,000] Accuracy: 94.40%, BalancedAccuracy: 93.05%, GeometricMean: 92.95%, CohenKappa: 87.34%	(rolling 500)
[2,500] Accuracy: 93.60%, BalancedAccuracy: 93.00%, GeometricMean: 92.98%, CohenKappa: 85.44%	(rolling 500)
[3,000] Accuracy: 94.00%, BalancedAccuracy: 92.06%, GeometricMean: 91.88%, CohenKappa: 86.18%	(rolling 500)
[3,500] Accuracy: 94.00%, BalancedAccuracy: 92.60%, GeometricMean: 92.53%, CohenKappa: 85.82%	(rolling 500)
[4,000] Accuracy: 95.80%, BalancedAccuracy: 95.23%, GeometricMean: 95.21%, CohenKappa: 90.30%	(rolling 500)
[4,500] Accuracy: 94.80%, BalancedAccuracy: 93.32%, GeometricMean: 93.21%, CohenKappa: 88.17%	(rolling 500)
[5,000] Accuracy: 93.40%, Bala

Accuracy: 76.80%, BalancedAccuracy: 75.23%, GeometricMean: 74.94%, CohenKappa: 50.67%	(rolling 500)